In [1]:
import datetime
import numpy as np 
import pandas as pd 
from ratelimit import limits, sleep_and_retry
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
from collections import Counter
import re
import csv
from nltk.tokenize import word_tokenize

In [5]:
# csv_file_path = "/Users/apple/PROJECT/Code_4_10k/nvidia_constituents.csv"
# columns = ["Name", "CIK", "Date", "Body" ]
# firms_df = pd.read_csv(csv_file_path)
# firms_df = firms_df.drop(['Security', 'GICS Sector', 'GICS Sub-Industry', 'Headquarters Location', 'Date added', 'Founded'], axis=1)
# firms_df['CIK'] = firms_df['CIK'].apply(lambda x: str(x).zfill(10))
# firms_dict = firms_df.set_index('Symbol')['CIK'].to_dict()
# type = '10-K'
# seen = set()
# firms_ciks = [cik for cik in firms_df['CIK'].tolist() if not (cik in seen or seen.add(cik))] 
# firms_ciks

['0001045810']

In [8]:
# data_raw_folder = f'/opt/airflow/data/{firms_ciks[0]}_{type}_html'
# sec10k_10k_extracted_folder = f'/opt/airflow/data/{firms_ciks[0]}_all_txt'
# sec10k_item1a_extracted_folder = f'/opt/airflow/data/{firms_ciks[0]}_riskFactor_txt' 
# print(sec10k_item1a_extracted_folder)
# print(sec10k_10k_extracted_folder)
# print(data_raw_folder)

/opt/airflow/data/0001045810_riskFactor_txt
/opt/airflow/data/0001045810_all_txt
/opt/airflow/data/0001045810_10-K_html


Get CIKs

In [2]:

QQQ_path = './sp500_total_constituents.csv'
# QQQ_path = './top10_QQQ_constituents.csv'

try:
    df = pd.read_csv(QQQ_path, encoding = 'utf-8')
    QQQ_cik = df['CIK'].drop_duplicates().tolist()
    QQQ_ticker = df['Symbol'].tolist()
    QQQ_cik_ticker = dict(zip(QQQ_cik, QQQ_ticker))
except UnicodeDecodeError:
    df = pd.read_csv(QQQ_path, encoding = 'ISO-8859-1')
    QQQ_cik = df['CIK'].drop_duplicates().tolist()
    QQQ_ticker = df['Symbol'].tolist()
    QQQ_cik_ticker = dict(zip(QQQ_cik, QQQ_ticker))




In [5]:
print(QQQ_cik_ticker)

{804328: 'QCOM', 49826: 'ITW', 715957: 'D', 1018963: 'ATI', 46080: 'HAS', 107263: 'WMB', 769397: 'ADSK', 32604: 'EMR', 795266: 'KBH', 909832: 'COST', 1568100: 'PD', 12927: 'BA', 23217: 'CAG', 87347: 'SLB', 101778: 'MRO', 1817358: 'ASO', 8818: 'AVY', 92380: 'LUV', 91440: 'SNA', 723125: 'MU', 1674862: 'ASH', 51253: 'IFF', 97476: 'TXN', 40545: 'GE', 38777: 'BEN', 318154: 'AMGN', 815097: 'CCL', 885639: 'KSS', 1012100: 'SEE', 1284812: 'CNS', 814453: 'NWL', 85961: 'R', 62709: 'MMC', 1881551: 'STI', 874766: 'HIG', 764622: 'PNW', 60667: 'LOW', 319201: 'KLAC', 37996: 'F', 823768: 'WM', 60519: 'LPX', 773840: 'HON', 827052: 'EIX', 1800: 'ABT', 876437: 'MTG', 70318: 'THC', 106640: 'WHR', 1164727: 'NEM', 827099: 'CPWR', 40987: 'GPC', 4962: 'AXP', 59558: 'LNC', 702165: 'NSC', 899051: 'ALL', 29905: 'DOV', 56873: 'KR', 49196: 'HBAN', 1666700: 'DD', 27419: 'TGT', 1031296: 'FE', 922224: 'PPL', 63276: 'MAT', 34088: 'XOM', 79879: 'PPG', 109198: 'TJX', 1390777: 'BK', 40704: 'GIS', 831001: 'C', 1474098: 'PE

In [11]:
q10_cik_ticker_prob = [ 813672,1109357,50863,1035267,1418135,1637459,319201,707549,1048286,1326801,865752,789019, 723125, 1065280, 1045810, 878927, 898173, 723531, 75362, 872589, 882835, 745732, 829224, 1318605, 1618921, 1437107, 1774585, 937966, 901832, 1650107, 1535527, 1792789, 1561550, 1709048, 1397187, 1441816, 1099590 ,1835632, 1327567, 1737806, 908937, 1353283, 1650372, 1671933, 1327811]
len(q10_cik_ticker_prob)

45

In [12]:
q10_QQQ_cik_ticker_prob = {cik: ticker for cik, ticker in QQQ_cik_ticker.items() if cik in q10_cik_ticker_prob }
q10_QQQ_cik_ticker_prob

{813672: 'CDNS',
 1109357: 'EXC',
 50863: 'ILMN',
 1035267: 'INTU',
 1418135: 'ISRG',
 1637459: 'KDP',
 319201: 'KHC',
 707549: 'KLAC',
 1048286: 'LRCX',
 1326801: 'MDLZ',
 865752: 'META',
 789019: 'MRNA',
 723125: 'MSFT',
 1065280: 'MU',
 1045810: 'NFLX',
 878927: 'NXPI',
 898173: 'ON',
 723531: 'ORLY',
 75362: 'PAYX',
 872589: 'QCOM',
 882835: 'REGN',
 745732: 'ROP',
 829224: 'ROST',
 1318605: 'TMUS',
 1618921: 'VRTX',
 1437107: 'WBA',
 1774585: 'XEL',
 937966: 'ABNB',
 901832: 'ASML',
 1650107: 'AZN',
 1535527: 'CCEP',
 1792789: 'CRWD',
 1561550: 'DASH',
 1709048: 'DDOG',
 1397187: 'GFS',
 1441816: 'LULU',
 1099590: 'MDB',
 1835632: 'MELI',
 1327567: 'MRVL',
 1737806: 'PANW',
 908937: 'PDD',
 1353283: 'SIRI',
 1650372: 'SPLK',
 1671933: 'TEAM',
 1327811: 'TTD'}

In [11]:
print(QQQ_cik)

[37303, 1159508, 1065521, 1117057, 1795079, 1575965, 312070, 1495219, 48465, 4962, 27904, 27419]


Download Reports

In [3]:
def submission_api(cik, ticker, doc_type, headers, start_date, end_date):
    # SEC submissions URL
    rss_url = f'https://data.sec.gov/submissions/CIK{cik}.json'

    # Retrieve the filing data from SEC
    sec_data = requests.get(url=rss_url, headers=headers)

    filings = sec_data.json().get('filings', {}).get('recent', {})

    entries = []

    # Iterate over the filings and filter by type and date range
    for i in range(len(filings['accessionNumber'])):
        filing_date = pd.to_datetime(filings['filingDate'][i])
        filing_type = filings['form'][i]


        if filing_type == doc_type and start_date <= filing_date <= end_date:

            accession_number = filings['accessionNumber'][i].replace('-', '')
            filing_href = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_number}/index.json"

            # Fetch the specific filing details
            filing_response = requests.get(filing_href, headers=headers)

            if filing_response.status_code == 200:
                filing_json = filing_response.json()
                for file in filing_json['directory']['item']:

                    if file['name'].endswith('.htm'):
                        if doc_type.lower() in file['name'] or '10k' in file['name'] or ticker.lower() in file['name']:
                            if 'ex' not in file['name']:
                                html_href = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_number}/{file['name']}"
                                entries.append((html_href, filing_type, filing_date))

                    
    return entries


In [4]:
import requests
import time
from tenacity import retry, stop_after_attempt, wait_fixed
import pandas as pd

class LimitRequest:
    SEC_CALL_LIMIT = {'calls': 10, 'seconds': 1}

    @retry(stop=stop_after_attempt(5), wait=wait_fixed(2))  # Retry up to 5 times with a 2-second delay
    def _call_sec(url, headers):
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response
        else:
            response.raise_for_status()  # Raise exception for failed requests

    @classmethod
    def get(cls, url, headers):
        return cls._call_sec(url, headers)



def get_sec_data(cik, ticker, doc_type, headers, end_date, start_date):


    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    
    # SEC XBRL data APIs
    xbrl_url = f'https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}/us-gaap/AccountsPayableCurrent.json'
    sec_data = requests.get(url=xbrl_url, headers=headers)
    print(sec_data)
    entries = []
    try: 
        units = sec_data.json().get('units', {}).get('USD', [])
    except (ValueError, KeyError, requests.exceptions.RequestException) as e:
        print(f"Error: {e}")
        try:
            return submission_api(cik, ticker, doc_type, headers, start_date, end_date)
        except Exception as e:
            print(f"Error: {e}")
    
    for i in range(len(units)):
        filing_date = pd.to_datetime(units[i]['filed'])
        filing_type = units[i]['form']
        filing_accn = units[i]['accn']
        
        if filing_type == doc_type.upper() and start_date <= filing_date <= end_date:

            filing_href = f"https://www.sec.gov/Archives/edgar/data/{cik}/{filing_accn.replace('-', '')}/index.json"
            filing_response = requests.get(filing_href, headers=headers)

            if filing_response.status_code == 200:
                filing_json = filing_response.json()
                for file in filing_json['directory']['item']:
                    if file['name'].endswith('.htm'):
                        if doc_type.lower() in file['name'] or "".join(doc_type.lower().split("-")) in file['name'] or ticker.lower() in file['name']:
                            if 'ex' not in file['name']:
                                html_href = f"https://www.sec.gov/Archives/edgar/data/{cik}/{filing_accn.replace('-', '')}/{file['name']}"

                                entries.append((html_href, filing_type, filing_date))

    entries = list(dict.fromkeys(entries))
    print('entries:', entries)
    return entries


            
        
    



In [5]:
def get_document_type(doc):
    """
    Return the document type lowercased

    Parameters
    ----------
    doc : str
        The document string

    Returns
    -------
    doc_type : str
        The document type lowercased
    """
    
    # Regex explaination : Here I am tryng to do a positive lookbehind
    # (?<=a)b (positive lookbehind) matches the b (and only the b) in cab, but does not match bed or debt.
    # More reference : https://www.regular-expressions.info/lookaround.html
    
    type_regex = re.compile(r'(?<=<TYPE>)\w+[^\n]+') # gives out \w
    type_idx = re.search(type_regex, doc).group(0)
    return type_idx

In [6]:
def get_document_format(doc):
    """
    Return the document type lowercased

    Parameters
    ----------
    doc : str
        The document string

    Returns
    -------
    doc_type : str
        The document type lowercased
    """
    
    format_regex = re.compile(r'(?<=<FILENAME>)\w+[^\n]+') # gives out \w
    doc_type  = re.search(format_regex, doc).group(0).lower()
    if doc_type.endswith((".htm", ".html")):
        return 'HTML'
    if doc_type.endswith(".txt"):
        return 'TXT'
    else:
        return None

In [7]:

def get_documents(text):
    document_start_regex = re.compile(r'<DOCUMENT>')
    document_end_regex = re.compile(r'<\/DOCUMENT>')
    
    document_start_indices = [match.start() for match in document_start_regex.finditer(text)]
    document_end_indices = [match.start() for match in document_end_regex.finditer(text)]
    
    documents = []
    for start_index, end_index in zip(document_start_indices, document_end_indices):
        document = text[start_index:end_index]
        documents.append(document)
        
    # If the filing is written in the XBRL content
    if not documents:
        # Parse the XBRL content
        documents.append(text)
    
    return documents

In [8]:
from tqdm import tqdm
def download_fillings(ciks_tickers, root_folder, doc_type, headers, end_date=datetime.datetime.now(), start_date = '1990-01-01'):
    
    for idx, (cik, ticker) in enumerate(ciks_tickers.items()):

        cik = str(cik).zfill(10)
        print("cik", cik)
        report_info = get_sec_data(cik, ticker, doc_type, headers, end_date=end_date, start_date=start_date)

        # check if 10-K exists, otherwise skip it
        if not report_info:
            print('hi')
            continue
        else:
            folder_path = os.path.join(root_folder, cik)
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

        for index_url, _ , file_date in tqdm(report_info, desc='Downloading {} Fillings'.format(cik), unit='filling'):
            file_date = file_date.strftime('%Y-%m-%d')


            file = LimitRequest.get(url=index_url, headers=headers)


            file_name = os.path.join(folder_path, file_date + '.html')
            with open(file_name,'w+') as f:
                f.write(file.text)
            f.close()


                # if get_document_type(document) == doc_type and get_document_format(document) == 'HTML':
                
                #     file_name = os.path.join(folder_path, file_date + '.html')
                #     with open(file_name,'w+') as f:
                #         f.write(document)
                #     f.close()
                # if get_document_type(document) == doc_type and get_document_format(document) == 'TXT':
                #     file_name = os.path.join(folder_path, file_date + '.txt')
                #     with open(file_name,'w+') as f:
                #         f.write(document)
                #     f.close()

Report starts from 2006; parts of 2005 reports don't have item1A <br>
Only for 10-k reports

In [9]:
root_folder = 'total_sp500_10q-html-2006'
doc_type = '10-Q'
headers = {'User-Agent': 'University of Edinburgh s2101369@ed.ac.uk'}
start_date = '2006-01-01',
end_date = datetime.datetime.now()
if not os.path.exists(root_folder):
    os.makedirs(root_folder)
download_fillings(QQQ_cik_ticker, root_folder,doc_type,headers,end_date=end_date,start_date=start_date)

cik 0000804328
<Response [200]>
entries: [('https://www.sec.gov/Archives/edgar/data/0000804328/000123445211000190/qcom3271110q.htm', '10-Q', Timestamp('2011-04-20 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000804328/000123445211000292/qcom6261110q.htm', '10-Q', Timestamp('2011-07-20 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000804328/000123445212000055/qcom12251110q.htm', '10-Q', Timestamp('2012-02-01 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000804328/000123445212000217/qcom3251210q.htm', '10-Q', Timestamp('2012-04-18 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000804328/000123445212000268/qcom6241210q.htm', '10-Q', Timestamp('2012-07-18 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000804328/000123445213000041/qcom12301210-q.htm', '10-Q', Timestamp('2013-01-30 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000804328/000123445213000179/qcom3311310-q.htm', '10-Q', Timestamp('2013-04-24 00:00:00')), ('https://www.se

KeyboardInterrupt: 

In [14]:
root_folder = 'total_sp500_10-Q'
doc_type = '10-Q'
headers = {'User-Agent': 'University of Edinburgh s2101369@ed.ac.uk'}
start_date = '2006-01-01',
end_date = datetime.datetime.now()
if not os.path.exists(root_folder):
    os.makedirs(root_folder)
download_fillings(QQQ_cik_ticker, root_folder,doc_type,headers,end_date=end_date,start_date=start_date)

cik 0000804328
<Response [200]>
entries: [('https://www.sec.gov/Archives/edgar/data/0000804328/000123445211000190/qcom3271110q.htm', '10-Q', Timestamp('2011-04-20 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000804328/000123445211000292/qcom6261110q.htm', '10-Q', Timestamp('2011-07-20 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000804328/000123445212000055/qcom12251110q.htm', '10-Q', Timestamp('2012-02-01 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000804328/000123445212000217/qcom3251210q.htm', '10-Q', Timestamp('2012-04-18 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000804328/000123445212000268/qcom6241210q.htm', '10-Q', Timestamp('2012-07-18 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000804328/000123445213000041/qcom12301210-q.htm', '10-Q', Timestamp('2013-01-30 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000804328/000123445213000179/qcom3311310-q.htm', '10-Q', Timestamp('2013-04-24 00:00:00')), ('https://www.se

cik 0000049826
<Response [200]>
entries: [('https://www.sec.gov/Archives/edgar/data/0000049826/000004982609000066/itw10q2q09.htm', '10-Q', Timestamp('2009-08-07 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000049826/000004982609000091/itw10q3q09.htm', '10-Q', Timestamp('2009-10-30 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000049826/000004982610000017/itw10q10.htm', '10-Q', Timestamp('2010-05-07 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000049826/000004982610000040/itw10q2q10.htm', '10-Q', Timestamp('2010-08-06 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000049826/000004982610000052/itw10q3q10.htm', '10-Q', Timestamp('2010-10-29 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000049826/000004982611000030/itw10q1q11.htm', '10-Q', Timestamp('2011-05-06 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000049826/000004982611000049/itw10q2q11.htm', '10-Q', Timestamp('2011-08-05 00:00:00')), ('https://www.sec.gov/Archives/edgar

cik 0000715957
<Response [200]>
entries: [('https://www.sec.gov/Archives/edgar/data/0000715957/000119312509220395/d10q.htm', '10-Q', Timestamp('2009-11-02 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000715957/000119312510099289/d10q.htm', '10-Q', Timestamp('2010-04-29 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000715957/000119312510173603/d10q.htm', '10-Q', Timestamp('2010-08-02 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000715957/000119312510241221/d10q.htm', '10-Q', Timestamp('2010-10-29 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000715957/000119312511117076/d10q.htm', '10-Q', Timestamp('2011-04-29 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000715957/000119312511201833/d10q.htm', '10-Q', Timestamp('2011-07-29 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000715957/000119312511285781/d247534d10q.htm', '10-Q', Timestamp('2011-10-28 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000715957/0001193125121854

cik 0001018963
<Response [200]>
entries: [('https://www.sec.gov/Archives/edgar/data/0001018963/000119312511210518/d10q.htm', '10-Q', Timestamp('2011-08-04 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001018963/000119312511297514/d249610d10q.htm', '10-Q', Timestamp('2011-11-04 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001018963/000119312512212043/d345708d10q.htm', '10-Q', Timestamp('2012-05-04 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001018963/000119312512334956/d387856d10q.htm', '10-Q', Timestamp('2012-08-03 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001018963/000119312512449198/d430842d10q.htm', '10-Q', Timestamp('2012-11-02 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001018963/000119312513201622/d531236d10q.htm', '10-Q', Timestamp('2013-05-06 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001018963/000119312513319181/d575027d10q.htm', '10-Q', Timestamp('2013-08-05 00:00:00')), ('https://www.sec.gov/Archives/edg

cik 0000046080
<Response [200]>
entries: [('https://www.sec.gov/Archives/edgar/data/0000046080/000004608011000083/q2201110q.htm', '10-Q', Timestamp('2011-07-29 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000046080/000004608013000041/q11310q.htm', '10-Q', Timestamp('2013-05-08 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000046080/000004608013000076/q210q2013.htm', '10-Q', Timestamp('2013-07-31 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000046080/000004608013000097/form10q3q.htm', '10-Q', Timestamp('2013-11-05 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000046080/000004608014000048/form10q1q.htm', '10-Q', Timestamp('2014-05-06 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000046080/000004608014000079/form10q2q.htm', '10-Q', Timestamp('2014-07-31 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0000046080/000004608014000109/form10q3q.htm', '10-Q', Timestamp('2014-11-05 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0

...

cik 0001687229
<Response [404]>
Error: Expecting value: line 1 column 1 (char 0)


cik 0001057352
<Response [200]>
entries: [('https://www.sec.gov/Archives/edgar/data/0001057352/000105735211000123/form10-q.htm', '10-Q', Timestamp('2011-07-28 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001057352/000105735211000139/form_10-q.htm', '10-Q', Timestamp('2011-10-27 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001057352/000105735212000063/csgp-3312012x10q.htm', '10-Q', Timestamp('2012-04-27 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001057352/000105735212000117/csgp-6302012x10q.htm', '10-Q', Timestamp('2012-07-26 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001057352/000105735212000158/csgp-9302012x10q.htm', '10-Q', Timestamp('2012-10-25 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001057352/000105735213000039/mastercopycsgp-3312013x10q.htm', '10-Q', Timestamp('2013-04-25 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001057352/000105735213000048/csgp-6302013x10q.htm', '10-Q', Timestamp('2013-07-24 00:00:00')

cik 0001389170
<Response [200]>
entries: [('https://www.sec.gov/Archives/edgar/data/0001389170/000138917011000037/form10-q.htm', '10-Q', Timestamp('2011-11-07 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001389170/000156459020037565/trgp-10q_20200630.htm', '10-Q', Timestamp('2020-08-06 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001389170/000156459020051164/trgp-10q_20200930.htm', '10-Q', Timestamp('2020-11-05 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001389170/000156459021024819/trgp-10q_20210331.htm', '10-Q', Timestamp('2021-05-06 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001389170/000156459021041376/trgp-10q_20210630.htm', '10-Q', Timestamp('2021-08-05 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001389170/000156459021054348/trgp-10q_20210930.htm', '10-Q', Timestamp('2021-11-04 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001389170/000156459022018168/trgp-10q_20220331.htm', '10-Q', Timestamp('2022-05-05 00:00:0

cik 0000947484
<Response [404]>
Error: Expecting value: line 1 column 1 (char 0)


cik 0001022671
<Response [200]>
entries: [('https://www.sec.gov/Archives/edgar/data/0001022671/000110465912035125/a12-6512_110q.htm', '10-Q', Timestamp('2012-05-09 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001022671/000110465912055077/a12-13653_110q.htm', '10-Q', Timestamp('2012-08-07 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001022671/000110465912075312/a12-19046_110q.htm', '10-Q', Timestamp('2012-11-07 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001022671/000110465913039480/a13-8325_110q.htm', '10-Q', Timestamp('2013-05-09 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001022671/000110465913060793/a13-13679_110q.htm', '10-Q', Timestamp('2013-08-07 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001022671/000110465913082749/a13-19580_110q.htm', '10-Q', Timestamp('2013-11-08 00:00:00')), ('https://www.sec.gov/Archives/edgar/data/0001022671/000110465914036418/a14-9194_110q.htm', '10-Q', Timestamp('2014-05-08 00:00:00')), ('https:/

# Trash

In [ ]:
# ?import requests
# import time
# from tenacity import retry, stop_after_attempt, wait_fixed
# import pandas as pd

# class LimitRequest:
#     SEC_CALL_LIMIT = {'calls': 10, 'seconds': 1}

#     @retry(stop=stop_after_attempt(5), wait=wait_fixed(2))  # Retry up to 5 times with a 2-second delay
#     def _call_sec(url, headers):
#         response = requests.get(url, headers=headers)
#         if response.status_code == 200:
#             return response
#         else:
#             response.raise_for_status()  # Raise exception for failed requests

#     @classmethod
#     def get(cls, url, headers):
#         return cls._call_sec(url, headers)



# def get_sec_data(cik, ticker, doc_type, headers, end_date, start_date, start, count):


#     start_date = pd.to_datetime(start_date)
#     end_date = pd.to_datetime(end_date)
    
#     # SEC submissions URL
#     rss_url = f'https://data.sec.gov/submissions/CIK{cik}.json'
#     print(rss_url)

#     # Retrieve the filing data from SEC
#     sec_data = requests.get(url=rss_url, headers=headers)

#     filings = sec_data.json().get('filings', {}).get('recent', {})


#     entries = []
#     # print('filings-form', filings['form'])
#     print('filing_date', filings['filingDate'][1000])
#     # print('doc_type', doc_type)
#     print('start_date', start_date)
#     print('end_date', end_date)
#     # Iterate over the filings and filter by type and date range
#     print('len', len(filings['accessionNumber']))
#     print('len-date', len(filings['filingDate']))

#     print('len-form', filings['form'][1000])

#     for i in range(len(filings['accessionNumber'])):
#         filing_date = pd.to_datetime(filings['filingDate'][i])
#         filing_type = filings['form'][i]

#         # if start_date <= filing_date <= end_date:


#         #     print('filing_date', filing_date)

#         if filing_type == doc_type and start_date <= filing_date <= end_date:


#             accession_number = filings['accessionNumber'][i].replace('-', '')
#             filing_href = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_number}/index.json"
#             print('filing_href', filing_href)
#             # Fetch the specific filing details
#             filing_response = requests.get(filing_href, headers=headers)

#             if filing_response.status_code == 200:
#                 filing_json = filing_response.json()
#                 for file in filing_json['directory']['item']:

#                     if file['name'].endswith('.htm'):
#                         if doc_type.lower() in file['name'] or '10k' in file['name'] or ticker.lower() in file['name']:  # Find HTML document
#                             html_href = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_number}/{file['name']}"
#                             # with open('output.txt', 'a') as f:
#                             #     f.write(html_href + "\n")

#                             entries.append((html_href, filing_type, filing_date))
#                             # break  # Stop after finding the HTML file
                    
#     return entries


In [ ]:
# class LimitRequest(object):
#     SEC_CALL_LIMIT = {'calls': 10, 'seconds': 1}
#     @sleep_and_retry
#     @limits(calls=SEC_CALL_LIMIT['calls'], period=SEC_CALL_LIMIT['seconds'])
#     def _call_sec(url,headers):
#         return requests.get(url,headers=headers)
    
#     @classmethod
#     def get(cls,url,headers):
#         return cls._call_sec(url, headers)

# def get_sec_data(cik, doc_type, headers, end_date, start_date, start, count):
#     start_date = pd.to_datetime(start_date)
#     end_date = pd.to_datetime(end_date)
    
#     # Updated SEC API URL
#     rss_url = f'https://data.sec.gov/submissions/CIK{cik}.json'
    
#     # Set proper headers to identify yourself to the SEC
#     sec_data = requests.get(url=rss_url, headers=headers)
#     filings = sec_data.json()['filings']['recent']

    
#     entries = []
    
#     # Loop through the filings and filter by the document type and date range
#     for i in range(len(filings['accessionNumber'])):
#         filing_date = pd.to_datetime(filings['filingDate'][i])
#         filing_type = filings['form'][i]
        
#         if filing_type == doc_type and start_date <= filing_date <= end_date:
#             filing_href = f"https://www.sec.gov/Archives/edgar/data/{cik}/{filings['accessionNumber'][i].replace('-', '')}/index.json"
#             entries.append((filing_href, filing_type, filing_date))
    
#     return entries

In [ ]:
# Oupdated API Call
# class LimitRequest(object):
#     SEC_CALL_LIMIT = {'calls': 10, 'seconds': 1}
#     @sleep_and_retry
#     @limits(calls=SEC_CALL_LIMIT['calls'], period=SEC_CALL_LIMIT['seconds'])
#     def _call_sec(url,headers):
#         return requests.get(url,headers=headers)
    
#     @classmethod
#     def get(cls,url,headers):
#         return cls._call_sec(url, headers)


# def get_sec_data(cik, doc_type, headers,end_date, start_date, start, count):
#     start_date = pd.to_datetime(start_date)
#     end_date = pd.to_datetime(end_date)
#     rss_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany' \
#         '&CIK={}&type={}&start={}&count={}&owner=exclude&output=atom' \
#         .format(cik, doc_type, start, count)
    
#     sec_data = LimitRequest.get(url = rss_url,headers=headers)
#     soup = BeautifulSoup(sec_data.content, 'xml')    
#     entries = [
#         (   entry.content.find('filing-href').getText(),
#             entry.content.find('filing-type').getText(),
#             entry.content.find('filing-date').getText())
#         for entry in soup.find_all('entry')
#         if pd.to_datetime(entry.content.find('filing-date').getText()) <= end_date and pd.to_datetime(entry.content.find('filing-date').getText()) >= start_date]  
#     return entries

In [ ]:
# from bs4 import BeautifulSoup
# def get_document_xml(text):
#     # Parse the XBRL content
#     soup = BeautifulSoup(text, 'lxml')
#     body_text = soup.body.get_text()
    
#     return body_text


# headers = {'User-Agent': 'University of Edinburgh s2101368@ed.ac.uk'}
# url = "https://www.sec.gov/Archives/edgar/data/0000320193/000032019323000106/aapl-20230930.htm"
# url2 = "https://www.sec.gov/Archives/edgar/data/0000320193/000032019317000070/a10-k20179302017.htm"
# file  = LimitRequest.get(url=url2, headers=headers)
# with open('output1.html', 'w') as f:
#     f.write(file.text)

# # for document in get_documents(file.text):
# #     with open('output5.html', 'w') as f:
# #         f.write(document)


# # Tester

# root_folder = 'data'
# doc_type = '10-K'
# headers = {'User-Agent': 'University of Edinburgh s2101368@ed.ac.uk'}
# start_date = '2006-01-01',
# end_date = datetime.datetime.now()
# for idx, (cik, ticker) in enumerate(QQQ_cik_ticker.items()):

#         cik = str(cik).zfill(10)

#         repo = get_sec_data(cik, ticker, doc_type, headers, end_date, start_date)
#         print(repo)